In [1]:
from scipy.fft import fft
import torch 
import pickle
import matplotlib.pyplot as plt

# Load the dataset

with open ('interaction_matrices.pkl', 'rb') as int_matrices:
    interaction_matrices = pickle.load(int_matrices)

with open ('spectra_dataset.pkl', 'rb') as spec_dataset:
    spec_dataset = pickle.load(spec_dataset)


# converting the dataset to numpy arrays

interaction_matrices = interaction_matrices.numpy()
spec_dataset = spec_dataset.numpy()

In [2]:
print(spec_dataset.shape)
print(interaction_matrices.shape)

# data is in good shape 
# NMF Y ~ WH
# Y is the dataset
# W IS THE BASIS MATRIX AND H IS THE COEFFICIENT MATRIX



(10000, 20000)
(10000, 6, 6)


In [3]:
from sklearn.decomposition import NMF
import numpy as np
import seaborn as sns

selected_spec = spec_dataset[:10000,:]
print(selected_spec.shape)

# W becomes the basis matrix holding the features i will use for the CNN
model = NMF(n_components=75, init='nndsvd', random_state=40, max_iter=200, beta_loss='frobenius')
W = model.fit_transform(selected_spec)
H = model.components_

approximation = np.dot(W,H)

reconstruction_error = np.linalg.norm(selected_spec - approximation, 'fro')
print("Forbenius norm error:", reconstruction_error)

(10000, 20000)


/Users/sanazkazeminia/anaconda3/envs/Mass_Spec/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Forbenius norm error: 0.0010816733


In [55]:
del model

In [45]:
print(H[2])
print(H.shape)

[0. 0. 0. ... 0. 0. 0.]
(75, 20000)


In [122]:
import torch 
import torch.nn.functional as Fun 
import numpy as np

# need to convert back to tensors then convert from float to int for one hot encoding

interaction_matrices = torch.tensor(interaction_matrices)
print(interaction_matrices.shape)
print(interaction_matrices.dtype)

interaction_matrices = interaction_matrices.to(torch.long)
print(interaction_matrices.dtype)
print(interaction_matrices.shape)

one_hot_encoding = Fun.one_hot(interaction_matrices, num_classes=7)
print(one_hot_encoding.shape)
print(one_hot_encoding.dtype)

spectra_data = torch.tensor(W)
print(spectra_data.shape)
print(spectra_data.dtype)


# attempting to split problem into 2 - first 0 or not then abundance.

# Convert to binary format where any presence of stoichiometry (non-zero class index) is marked as 1

# Step 1: Determine if the class index is greater than 0 (absence is class 0)
binary_matrices = torch.argmax(one_hot_encoding, dim=-1)  # Convert one-hot to indices [N, 6, 6]
binary_matrices = (binary_matrices > 0).float()  # Convert indices to binary (1 if presence, 0 if absence)
print(f"the binary matrices dtype is {binary_matrices.dtype}")

print("Binary Targets Shape:", binary_matrices.shape)  # Should be [N, 6, 6]
print("Sample Binary Target:", binary_matrices[0])  # Display the first converted binary target matrix
print(interaction_matrices[0])  # Display the original interaction matrix



torch.Size([10000, 6, 6])
torch.int64
torch.int64
torch.Size([10000, 6, 6])
torch.Size([10000, 6, 6, 7])
torch.int64
torch.Size([10000, 75])
torch.float32
the binary matrices dtype is torch.float32
Binary Targets Shape: torch.Size([10000, 6, 6])
Sample Binary Target: tensor([[0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [1., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 1.],
        [0., 0., 1., 1., 1., 0.],
        [0., 1., 0., 0., 1., 0.]])
tensor([[0, 0, 0, 5, 0, 0],
        [2, 0, 0, 0, 5, 0],
        [1, 0, 3, 2, 5, 5],
        [2, 2, 1, 3, 0, 6],
        [0, 0, 3, 3, 1, 0],
        [0, 2, 0, 0, 5, 0]])


/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_10776/739251209.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  interaction_matrices = torch.tensor(interaction_matrices)


In [126]:
from sklearn.model_selection import train_test_split
spectra_data = spectra_data.float()
print(spectra_data.dtype)
#binary is the matrices one hot encoded and the spec_train and test is W.
# to return to V we can multiply W by H to get the approximation of the original dataset
spec_train, spec_test, binary_train, binary_test = train_test_split(spectra_data, binary_matrices, test_size=0.2, random_state=42)
spec_test, spec_val, binary_test, binary_val = train_test_split(spec_test, binary_test, test_size=0.5, random_state=42)


torch.float32


In [127]:
print(spec_train.dtype)
print(binary_train.dtype)


torch.float32
torch.float32


In [128]:
import torch
import torch.nn as nn
import torch.optim as optim

class SpectraCNN(nn.Module):
    def __init__(self):
        super(SpectraCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.25)
        
        # Fully connected layers
        # Calculate the size after convolutions and pooling
        self.fc1 = nn.Linear(32 * 18, 128)  # Adjust the size according to your exact dimensions
        self.fc2 = nn.Linear(128, 36)  # Output 36 units, one for each position in the 6x6 matrix

        # Activation function
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten the convolutions
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)  # Sigmoid activation to output probabilities between 0 and 1
        return x.view(-1, 6, 6)  # Reshape to 6x6 matrix for each sample in the batch

# Instantiate the model
model = SpectraCNN()
print(model)


SpectraCNN(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=576, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=36, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [141]:
from torch.utils.data import DataLoader, TensorDataset

spec_train = spec_train.squeeze()  # This will remove all singleton dimensions
print(spec_train.shape)

spec_train = spec_train.unsqueeze(1)  # This will add a singleton dimension at the second position
print(spec_train.shape)
spec_train = torch.tensor(spec_train, dtype=torch.float32)
print(spec_train.dtype)

train_dataset = TensorDataset(spec_train, binary_train)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

spec_train.dtype
binary_train.dtype

torch.Size([8000, 75])
torch.Size([8000, 1, 75])
torch.float32


/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_10776/2217302572.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spec_train = torch.tensor(spec_train, dtype=torch.float32)


torch.float32

In [142]:
# Initialize model, optimizer, and loss function
model = SpectraCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

# Assuming your DataLoader setup is correct
# Train model
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    for spectra, matrices in train_loader:
        # Forward pass
        outputs = model(spectra)
        #print(outputs[0:4])
        # Ensure target is of the correct shape, with no channel dimension and type long
        matrices = matrices.long()
        loss = criterion(outputs, matrices.float())  # outputs should be [N, C, d1, d2], matrices should be [N, d1, d2]
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/50], Loss: 0.6932
Epoch [2/50], Loss: 0.6931
Epoch [3/50], Loss: 0.6932
Epoch [4/50], Loss: 0.6932
Epoch [5/50], Loss: 0.6931
Epoch [6/50], Loss: 0.6932
Epoch [7/50], Loss: 0.6931
Epoch [8/50], Loss: 0.6931
Epoch [9/50], Loss: 0.6931
Epoch [10/50], Loss: 0.6931
Epoch [11/50], Loss: 0.6931
Epoch [12/50], Loss: 0.6931
Epoch [13/50], Loss: 0.6932
Epoch [14/50], Loss: 0.6931
Epoch [15/50], Loss: 0.6931
Epoch [16/50], Loss: 0.6931
Epoch [17/50], Loss: 0.6931
Epoch [18/50], Loss: 0.6930
Epoch [19/50], Loss: 0.6932
Epoch [20/50], Loss: 0.6932
Epoch [21/50], Loss: 0.6931
Epoch [22/50], Loss: 0.6930
Epoch [23/50], Loss: 0.6931
Epoch [24/50], Loss: 0.6931
Epoch [25/50], Loss: 0.6931
Epoch [26/50], Loss: 0.6931
Epoch [27/50], Loss: 0.6932
Epoch [28/50], Loss: 0.6930
Epoch [29/50], Loss: 0.6931
Epoch [30/50], Loss: 0.6930
Epoch [31/50], Loss: 0.6928
Epoch [32/50], Loss: 0.6930
Epoch [33/50], Loss: 0.6930
Epoch [34/50], Loss: 0.6929
Epoch [35/50], Loss: 0.6931
Epoch [36/50], Loss: 0.6931
E

tensor([[[0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 1., 1., 1.],
         [0., 1., 1., 1., 1., 1.],
         [0., 1., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 1., 1., 0.],
         [0., 0., 0., 1., 1., 0.],
         [1., 1., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 1., 1.],
         [0., 1., 1., 0., 0., 1.]],

        [[0., 1., 0., 0., 0., 0.],
         [1., 0., 1., 1., 1., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 1., 0., 1.],
         [0., 0., 1., 1., 0., 0.],
         [0., 0., 0., 1., 1., 0.]],

        ...,

        [[0., 0., 1., 1., 0., 1.],
         [1., 0., 1., 0., 1., 0.],
         [1., 1., 1., 0., 0., 0.],
         [1., 0., 1., 0., 1., 1.],
         [1., 1., 0., 1., 0., 1.],
         [0., 0., 0., 1., 0., 1.]],

        [[0., 1., 1., 1., 1., 0.],
         [0., 0., 0., 0., 1., 0.],
         [1., 0., 1., 0., 1., 1.],
         [0., 0., 1., 0., 1., 0.]

Converted Target Indices Shape: torch.Size([10000, 6, 6])
